In [ ]:
!git clone https://github.com/erhancanozcan/prot_opt.git
#check beta distribution to set dual variables.

In [ ]:
#necessary imports
import torch
from torch.optim import Optimizer
import os
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
import random
from numpy import inf
import pandas as pd


from prot_opt.code.xor_data_generate import produce_xor_and_candidates
from prot_opt.code.xor_data_generate import plot_xor
from prot_opt.code.xor_data_generate import generate_dual_pns
from prot_opt.code.true_objective import true_loss
from prot_opt.code.true_objective import estimated_loss
from prot_opt.code.objective_plot import contour_plot 
from prot_opt.code.available_routines import routines

In [ ]:
candidates,df_class =produce_xor_and_candidates()
f,ax= plot_xor(df_class)

In [ ]:
dist_names=['bernoulli_5']
seed_list=np.arange(10)

optimizers=["SGD_c_l_rate","SGD_scale_by_ep","SGD_adaptive","SGD_with_Momentum",\
            "Adagrad","Adam_sg","Adam_full_batch","GD_c_l_rate","GD_scale_by_ep","SVRG"]
    
    

naive_initialization=[True,False]

learning_rate=[0.00001,0.0001,0.001,0.01,0.1]

epoch=[10]
#epoch=[20,50,100]

momentum_list=[0.1]


df = pd.DataFrame(columns=['Dual_dist','seed','Algorithm','naive_init','lr','epoch_num'\
                           ,'momentum','best_obj','max_grid_obj','max_data_obj'])


cntr=0
for dist in dist_names:
    for seed in seed_list:
        #genereta dual coefficients:
        df_class,pos,neg,d_of_data,num_pairs,dual_pns=generate_dual_pns(df_class,dist,seed)
        X,Y,Z=contour_plot(candidates,pos,neg,dual_pns)

        #different initialization strategies: maximizier_data_point, maximizer_grid_point
        Z_data_points=contour_plot(df_class[:,:2],pos,neg,dual_pns,no_grid=True)
        
        maximizer_data_point=df_class[np.argmax(Z_data_points),:2]
        maximizer_grid_point=np.zeros(d_of_data,int)
        for i in range (len(np.where(Z==np.max(Z)))):
            maximizer_grid_point[i]=int((np.where(Z==np.max(Z)))[i][0])
        maximizer_grid_point=np.array([X[maximizer_grid_point[0],maximizer_grid_point[1]],Y[maximizer_grid_point[0],maximizer_grid_point[1]]])
        
        data_point_obj=-np.max(Z_data_points)
        grid_point_obj=-np.max(Z)
        for opt in optimizers:
            for nv_init in naive_initialization:
                for lr in learning_rate:
                    for ep in epoch:
                        if opt=="SGD_with_Momentum":
                            for momentum in momentum_list:
                                routine1=routines(opt,l_rate=lr,epoch_no=ep,dual_pns=dual_pns,pos=pos,neg=neg,d_of_data=d_of_data,naive_initialization=nv_init,init_point=maximizer_data_point,betas=(0.9,0.999),alpha=momentum)
                                s_trajectory1,cumul_loss1,s_trajectory2,cumul_loss2,best_s,tmp_min=routine1.run()
                                tmp={'Dual_dist':dist,'seed':seed,'Algorithm':opt,\
                                     'naive_init':nv_init,'lr':lr,'epoch_num':ep\
                                      ,'momentum':momentum,'best_obj':tmp_min,\
                                     'max_grid_obj':grid_point_obj,'max_data_obj':data_point_obj}
                                print(tmp)
                                df = df.append(tmp, ignore_index = True)
                                #print(cntr)
                                cntr+=1
                        else:
                            routine1=routines(opt,l_rate=lr,epoch_no=ep,dual_pns=dual_pns,pos=pos,neg=neg,d_of_data=d_of_data,naive_initialization=nv_init,init_point=maximizer_data_point,betas=(0.9,0.999),alpha=0)
                            s_trajectory1,cumul_loss1,s_trajectory2,cumul_loss2,best_s,tmp_min=routine1.run()
                            tmp={'Dual_dist':dist,'seed':seed,'Algorithm':opt,\
                                     'naive_init':nv_init,'lr':lr,'epoch_num':ep\
                                      ,'momentum':'NA','best_obj':tmp_min,\
                                     'max_grid_obj':grid_point_obj,'max_data_obj':data_point_obj}
                            print(tmp)
                            df = df.append(tmp, ignore_index = True)
                            #print(cntr)
                            cntr+=1
                            

from google.colab import files

df.to_csv('df.csv')
files.download('df.csv')                        
                        

In [ ]:
np.arange(10,20,1)